# Prieskumná (Exploratívna) analýza dát
Dáta predstavujú záznamy o pacientoch s chorobou štítnej žľazy. Cieľom je vedieť predikovať hodnotu Y. Cieľom tejto časti je zanalytovanie datasetu. Využívame exploratívnu analýzu, deskriptívnu štatistiku a rôzne podporné vizualizácie. Poukazujeme na viaceré problémy, ktoré sa v dátach
nachádzajú (formáty dát, chýbajúce, nezmyselné alebo vychýlené hodnoty a pod.). 

In [466]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn
from sklearn import linear_model as lm

In [467]:
import dateutil.parser
import datetime

In [468]:
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (8, 6)
from IPython.display import Image, SVG, display

**AKCIA:** Vypíšeme si časť surových dát aby sme skontrolovali, či majú správny formát csv súboru.

In [29]:
%%bash 
head data/zadanie4/test.csv

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,query hyperthyroid,lithium,goitre,tumor,hypopituitary,psych,TSH measured,TSH,T3 measured,T3,TT4 measured,TT4,T4U measured,T4U,FTI measured,FTI,TBG measured,TBG,referral source,class,fnlwgt,education,education-num,relationship,capital-gain,capital-loss,hours-per-week,date_of_birth,personal_info
0,68,M,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,3.0,t,2.6,t,117.0,t,1.24,t,94,f,?,SVI,negative.|1231,96245, HS-grad,9, Husband,0.0,0,99,1949-07-30,"Sales|United-States
Married-civ-spouse -- Self-emp-inc|White"
1,42,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,1.8,t,2.0,t,66.0,t,0.93,t,71,f,?,SVHC,negative.|329,191177, HS-grad,9, Not-in-family,,0,55,1975-10-31,"Exec-managerial|United-States
Never-married -- Private|White"
2,27,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,2.2,t,2.3,t,105.0,t,0.97,t,109,f,?,other,negative.|2518,48585, Some-college,1000, Wife,0.0,0,4,1990-10-29 00:00:00,"Prof-specialty|United

**POZOROVANIE:** Z prvých riadkov súboru sa zdá, že **atribúty** a ich hodnoty sú **odelené čiarkami** a **záznamy** sú **odelené znakom nového riadku**. Prvý riadok začína čiarkou, čo môže znamenať, že v súbore chýba názov prvého atribútu. Z hodnôt prislúchajúcich prvému atribútu odhadujeme, že sa jedná o **identifikačné číslo**.

**AKCIA:** Načítame si dataset s trénovacími dátami a vypíšeme si základnú informáciu o počte atribútov (stĺpcov) a počte záznamov (riadkov)

**KOREKCIA:** Prvý stlpec si zadefinujeme ako index

In [375]:
file_path = "data/zadanie4/train.csv"
#file_path = "data/zadanie4/test.csv"
# Načítame si súbor s datasetom so separátom ',' a ako index nastavime prvý stĺpec
data = pd.read_csv(file_path, sep=',', index_col=0)
# Pomenujeme si index
data.index.name = 'id'

In [372]:
# Vypíšeme si základné informácie o datasete
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2800 entries, 0 to 2799
Data columns (total 39 columns):
age                          2799 non-null float64
sex                          2800 non-null object
on thyroxine                 2800 non-null object
query on thyroxine           2800 non-null object
on antithyroid medication    2800 non-null object
sick                         2800 non-null object
pregnant                     2800 non-null object
thyroid surgery              2800 non-null object
I131 treatment               2800 non-null object
query hypothyroid            2800 non-null object
query hyperthyroid           2800 non-null object
lithium                      2800 non-null object
goitre                       2800 non-null object
tumor                        2800 non-null object
hypopituitary                2800 non-null object
psych                        2800 non-null object
TSH measured                 2800 non-null object
TSH                          2516 non-null

**POZOROVANIE:** Z týchto základných informácií môžeme pozorovať, že niektoré atribúty obsahujú aj **null hodnoty**. Pre atribúty **age, TSH, T3, TT4, T4U, capital-gain, date_of_birth** budeme musiet null hodnoty vhodne doplniť alebo vyradiť celý záznam z datasetu.
Na prvý pohľad nazvy niektorých atribútov *(napr. T3, TT4, T4U atď.)* nevieme čo znamenajú a preto bude potreba si naštudovať doménovú oblasť a pokusiť sa ich **lepšie zadefinovať**. Názvy atribútov tiež niesu konzistentné. Pokiaľ sa atribút skladá z viacerých slov *(napr. pomlčka, medzera, počiarkovník)*.  
**Počet riadkov:** 2800   
**Počet stĺpcov:** 40  
**Dátové typy:** float64(6), int64(4), object(29)  

*POZNÁMKA: Doménový opis dát a ich definície sú v súbore data_description.txt*

# Analyza atribútov (stĺpcov)
V tejto časti si popíšeme každý jeden atribút osobitne.

### Atribút - age

In [6]:
# Výpis prvých 5 záznamov pre daný atribút
data['age'].head()

id
0    72.0
1    52.0
2    62.0
3    23.0
4    63.0
Name: age, dtype: float64

**POZOROVANIE:** Kedže "age" v preklade znamená **vek** a tento atribút v sebe nesie číselné hodnoty usudzujeme, že nám **hovorí o veku pacientov**. Vek budeme považovat za **spojitý (kvantitatívny) intervalový atribút** (To znamená, že nad ním môžeme vykonať výpočtovú deskriptívnu štatistiku.)

In [7]:
# vykonáme deskriptívnu štatistiku
data['age'].describe()

count    2799.00000
mean       51.84423
std        20.46116
min         1.00000
25%        36.00000
50%        54.00000
75%        67.00000
max       455.00000
Name: age, dtype: float64

**POZOROVANIE:** Jeden pacient **nemá zaznamenaný údaj o veku** a aspoň jeden pacient má vyplnený nezmyselný **vek 455 rokov**. Takéto nezmyselné hodnoty môžeme odstrániť alebo doplniť na základe priemerného veku pacientov **52 rokov**.

**KOREKCIA:** Vyfiltrujeme si hodnoty, ktoré potrebujeme zmeniť a vypočítame hodnotu **veku**. V prípade ak by boli dáta zaznemané v rovnakom roku boli by sme schopní vypočítať tento rok ako "rok narodenia" + "prislúchajúci vek" a **chýbajúce hodnoty** pre **vek** podľa atribútu "vypočítaný rok" - "rok narodenia". Z takto upraveného datasetu by sme museli dodatočne vyfiltrovať nezmyselné hodnoty a nahradiť ich za **priemerný vek** pacientov. V prípade ak dáta nie sú z jedného roku, doplníme chýbajúci vek **priemerným vekom** všetkých pacientov v datasete.

In [8]:
# Zistíme všetky hodnoty, ktoré sa nám zdajú nezmyslené alebo chýbajú
data[(data['age'] < 1) | (data['age'] > 100) | (pd.isnull(data['age']))]

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,...,class,fnlwgt,education,education-num,relationship,capital-gain,capital-loss,hours-per-week,date_of_birth,personal_info
id,,,,,,,,,,,,,,,,,,,,,
476,NaN,F,t,f,f,f,f,f,t,f,...,negative.|1,122975,HS-grad,9,Wife,0.0,0,21,??,Machine-op-inspct|Trinadad&Tobago\nMarried-civ...
685,455.0,F,f,f,f,f,f,f,f,f,...,negative.|861,151474,9th,5,Unmarried,0.0,0,40,1563-01-24,Sales|United-States\nDivorced -- Private|White


In [9]:
# Zistíme si mode (modus): najčastejšia hodnota (hodnota s najvačšou pravdepodobnostou vyskytu)
data['age'].mode()

0    59.0
dtype: float64

**POZOROVANIE:** Môžeme si všimnúť, že väčšina pacientov v datasete je vo veku **59 rokov**.

### Atribút - date_of_birth

In [159]:
# Výpis prvých 5 záznamov pre daný atribút
data['date_of_birth'].head(20)

id
0     1946-02-03
1     1965-08-27
2     1955-04-15
3     1994-03-26
4     1955-01-21
5     1961-12-10
6     1934-03-03
7     1949-07-21
8     1999-01-06
9     1945-11-10
10    1999-09-20
11           NaN
12           NaN
13    1970-09-28
14    1942-08-15
15    07/07/1973
16    1979-12-04
17    18/01/1940
18    15/10/1952
19    1981-07-19
Name: date_of_birth, dtype: object

**POZOROVANIE:** Kedže **date_of_birth** v preklade znamená **dátum narodenia** a tento atribút v sebe nesie rôzne dátumy usudzujeme, že nám **hovorí o dátume narodenia pacientov**. Dátum narodenia budeme považovat za **spojitý (kvantitatívny) intervalový atribút**. Hodnoty atribútu majú **rôzny formát** a obsahuju aj **nan** hodnoty.

**KOREKCIA:** Ujednotenie formátu dátumov.

In [306]:
# Ujednotíme formát dátumov a všetky hodnoty z ktorých nebude možné vytvoriť date object preved na NaT hodnotu
data['clean_date'] = pd.to_datetime(data['date_of_birth'], errors='coerce')
# Vypíšeme si štatistiku
data['clean_date'].describe()
# Vyfiltrujeme si nezmyslený dátum
# data[data['clean_date'] == '2067-09-01 00:00:00']

count                    2362
unique                   2256
top       1983-08-20 00:00:00
freq                        3
first     1923-05-30 00:00:00
last      2067-09-01 00:00:00
Name: clean_date, dtype: object

**POZOROVANIE:** Po zjednoteni datumov možeme vidieť, že v dátach sú pacienti, ktorí sa ešte nenarodili **2067-09-01**. Táto hodnota je zlá lebo pôvodná hodnota v subore.csv bola **67-09-01**, čo pravdepodobne predstavovalo dátum **1967-09-01**. Kedže funckia to_datetime má svoje limitácie [http://pandas-docs.github.io/pandas-docs-travis/timeseries.html#timestamp-limitations]navrhujeme] v ďaľšej časti navrhujeme odstrániť tieto dátumy. 

**KOREKCIA:** Vlastná funkcionalita pre uprávu formátov dátumov. 

In [308]:
# kvoli http://pandas-docs.github.io/pandas-docs-travis/timeseries.html#timestamp-limitations
# func konvertuje string na datetime
# Otazka je ci to potrebujeme
def my_to_datetime(val):
    if pd.notnull(val) and type(val) is str:
        val = (val.strip())[0:10]
        try:
            #return pd.to_datetime(str, errors='coerce')
            # Keď nedokáže naparsovat string raisne exception
            result = dateutil.parser.parse(val)
            return result
        except ValueError:
            # Pokusi sa znova rozparsovat string na datum a v pripade ak sa jej to nepodari nastavi NaT
            result = pd.to_datetime(str, errors='coerce')
            return result
    else:
        result = pd.to_datetime('today')
        return result
    
    
    
    

In [333]:
# Vytvoríme si stĺpec s hodnotami aj ktore su v zlom formate
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.select_dtypes.html
# data.loc[:'my_to_datetime',:].select_dtypes(include=[np.datetime64])
data['my_to_datetime'] = data['date_of_birth'].apply(my_to_datetime)
data['my_to_datetime'].describe()


count                    2710
unique                   2400
top       2018-03-18 00:00:00
freq                      190
Name: my_to_datetime, dtype: object

### Atribút - sex

In [ ]:
# Výpis prvých 5 záznamov pre daný atribút
data['sex'].head()

**POZOROVANIE:** Kedže "sex" v preklade znamená **pohlavie** a na prvý pohľad vidíme hodnoty F *(female - žena)*, M *(male - muž)* usudzujeme, že nám hovorí o **pohlaví pacientov**. Pohlavie budeme považovat za **kategorický nominálny atribút**. 

*POZNÁMKA: Nemci uzákonili aj tretie pohlavie "intersex" [http://www.independent.co.uk/news/world/europe/germany-third-gender-male-female-intersex-court-parliament-bundesverfassungsgericht-berlin-lgbt-a8043261.html]*

In [ ]:
# zistíme si základné charakteristiky záznamov
data['sex'].describe()

**POZOROVANIE:** V datasete sa nachádzajú **3 typy pohlaví**. Pacienti sú väčšinou **ženy**.

In [331]:
# Vypíšeme si unikátne hodnoty daného atribútu
data['sex'].unique()
data['sex'].value_counts()

F    1830
M     860
?     110
Name: sex, dtype: int64

**POZOROVANIE:** Dataset obsahuje tri pohlavia *F (female - žena), M (male - muž), **?** (chyba alebo intersex)*. Ak chýbajúcu hodnotu nebudeme považovať za tretie pohlavie budeme ju musieť vyzistiť z dalších atribútov.

In [ ]:
#Vyfiltrujeme si pacientov s nešpecifikovaným pohlavím.
data[(data['sex'] == '?')].head()

**POZOROVANIE:** Chýbajúcu hodnotu môžeme zistiť napríklad z atribútu **pregnant** alebo z atribútu **relationship**  

**KOREKCIA:** Vyfiltrujeme si pacientov, ktorých pohlavie je neznáme a sú tehotný. Ak je **pacient tehotný, potom je to žena**.

In [ ]:
# Vyfiltrujeme si pacientov s nešpecifikovaným pohlavím, ktorá sú tehotní.
data[(data['sex'] == '?') & (data['pregnant'] == 't')].head()

In [ ]:
#Neexistuje v datasete muž/žena, ktorý/ktorá je vydatý/oženená.
data[(data['sex'] == 'M') & (data['relationship'] == ' Wife')]
#data[(data['sex'] == 'F') & (data['relationship'] == ' Husband')]

**POZOROVANIE:** Kedže neexistuje v datasete vydatý muž a oženená žena usudzujeme že podľa atribútu **relationship** môžeme určiť pohlavie.

**KOREKCIA:**  Ak je pacient vo **vzťahu vydatá** *(Wife)*, potom je **ženského pohlavia**. Ak je pacient vo vzťahu **oženený** *(Husband)*, potom je **mužského pohlavia**. 


### Atribút - hours-per-week

In [322]:
# Výpis prvých 5 záznamov pre daný atribút
data['hours-per-week'].head()

id
0    40
1    38
2    45
3    20
4    40
Name: hours-per-week, dtype: int64

**POZOROVANIE:** Niesme si istí, čo znamená tento atribút."hours-per-week" sa používa v spojení **vykonávať činnosť určitý počet hodín v týždni**. Vyzerá to, že hodnoty sú reprezentované v hodinách. Budeme ho považovať za **spojitý (kvantitatívny) intervalový atribút** a preto na ňom vykonáme štatistiku.

In [339]:
# Výpis prvých 5 záznamov pre daný atribút
data['hours-per-week'].describe()

count    2800.000000
mean       38.544643
std        12.039004
min         1.000000
25%        35.000000
50%        40.000000
75%        40.000000
max        99.000000
Name: hours-per-week, dtype: float64

**POZOROVANIE:** Keď sa pozrieme na priemernú hodnotu **38.54** predpokladáme, že sa jedná o **pracovný čas** každého pacienta. Náš predpoklad je založený na tom, že bežná pracovná doba je 8 hodín denne 5 dní v týždni, čo sa rovná **40 hodinám**. Tieto dáta neobsahujú **žiadne null hodnoty** ani abnormálne výchylky a preto ich čistenie nie je potrebé.

## Liečba

### Atribút - on thyroxine

In [326]:
# Výpis prvých 5 záznamov pre daný atribút
data['on thyroxine'].head()

id
0    t
1    f
2    F
3    f
4    t
Name: on thyroxine, dtype: object

**POZOROVANIE:** Tyroxin je hormón, ktorý je produkovaný štítnou žľazou.[https://www.newsmax.com/fastfeatures/thyroid-thyroxine/2016/03/24/id/720782/]. V prípade zníženej produkcie tohto hormónu sa predpisuje vo forme lieku. Usudzujeme, že tento atribút nám hovorí o tom, čí daný pacient užíva liek obsahujúci hormón tyroxín. Existujú len dva stavy. Buď pacient užíva liek alebo neužíva. Preto budeme tento atribút považovat za **kategorický binárny atribút**, ktorý v sebe bude niesť hodnoty [true = užíva liek, false = neuziva liek].

In [340]:
# data['on thyroxine'].unique()
data['on thyroxine'].value_counts()

f        2218
t         302
FALSE     129
F         123
TRUE       18
T          10
Name: on thyroxine, dtype: int64

**POZOROVANIE:** Vidíme, že náš predpoklad bol správny a hodnoty pre atribút majú iba dva stavy (true, false). Tieto dáta bude potreba **očistiť na jednotný formát**

**KOREKCIA:** Ujednotíme formát dát

In [401]:
# Mapovanie True a False na hodnoty podľa toho ake si zvolíme
data['_on_thyroxine'] = data['on thyroxine'].map({'t': True, 'T': True, 'TRUE': True, 'f': False, 'F': False, 'FALSE': False})
data['_on_thyroxine'].value_counts()

False    2470
True      330
Name: on_thyroxine, dtype: int64

### Atribút - query on thyroxine

In [399]:
# Výpis prvých 5 záznamov pre daný atribút
data['query on thyroxine'].value_counts()

f    2760
t      40
Name: query on thyroxine, dtype: int64

**POZOROVANIE:** Niesme si istý co predstavuje tento atribút.  Usudzujeme, že tento atribút nám hovorí o tom, čí sa daný pacient sťažoval liek obsahujúci hormón tyroxín. Existujú len dva stavy. Buď sa pacient sťažoval liek alebo nesťažoval. Preto budeme tento atribút považovat za **kategorický binárny atribút**, ktorý v sebe bude niesť hodnoty [true = sťažoval sa/vypytoval, false = nesťažoval/nevypytoval].

**KOREKCIA:** Prekonvertujeme si formát dát na boolean hodnoty

In [402]:
# Mapovanie True a False na hodnoty podľa toho ake si zvolíme
data['_query_on_thyroxine'] = data['query on thyroxine'].map({'t': True, 'f': False})
data['_query_on_thyroxine'].value_counts()

False    2760
True       40
Name: query_on_thyroxine, dtype: int64

### Atribút - on antithyroid medication

In [437]:
# Výpis prvých 5 záznamov pre daný atribút
data['on antithyroid medication'].value_counts()

f    2766
t      34
Name: on antithyroid medication, dtype: int64

**POZOROVANIE:** Antithyroid je droga, ktorá znižuje funkciu štítnej žlazy. Používa sa v prípade ochorenia hypertireózy.[https://www.medicinenet.com/script/main/art.asp?articlekey=15316] Usudzujeme, že tento atribút nám hovorí o tom, čí daný pacient užíva drogu. Existujú len dva stavy. Buď pacient užíva drogu alebo neužíva. Preto budeme tento atribút považovat za **kategorický binárny atribút**, ktorý v sebe bude niesť hodnoty [true = užíva drogu, false = neuziva drogu].

**KOREKCIA:** Prekonvertujeme si formát dát na boolean hodnoty

In [438]:
# Mapovanie True a False na hodnoty podľa toho ake si zvolíme
data['_on_antithyroid'] = data['on antithyroid medication'].map({'t': True, 'f': False})
data['_on_antithyroid'].value_counts()

False    2766
True       34
Name: _on_antithyroid, dtype: int64

### Atribút - I131 treatment

In [443]:
# Výpis unikátnych hodnôt
data['I131 treatment'].value_counts()

f    2752
t      48
Name: I131 treatment, dtype: int64

**POZOROVANIE:** Radioaktívna, ktorá znižuje funkciu štítnej žlazy. [https://www.radiologyinfo.org/en/info.cfm?pg=radioiodine] Používa sa v prípade **ochorenia hypertireózy**. Usudzujeme, že tento atribút nám hovorí o tom, čí daný pacient podstupuje liečbu. Existujú len dva stavy. Buď pacient podstupuje liečbu alebo nepodstupuje. Preto budeme tento atribút považovat za **kategorický binárny atribút**, ktorý v sebe bude niesť hodnoty [true = podstupuje liečbu, false = nepodstupuje liečbu].

**KOREKCIA:** Prekonvertujeme si formát dát na boolean hodnoty

In [441]:
# Mapovanie True a False na hodnoty podľa toho ake si zvolíme
data['_I131_treatment'] = data['I131 treatment'].map({'t': True, 'f': False})
data['_I131_treatment'].value_counts()

False    2752
True       48
Name: _I131_treatment, dtype: int64

### Atribút - lithium

In [446]:
# Výpis unikátnych hodnôt
data['lithium'].value_counts()

f    2786
t      14
Name: lithium, dtype: int64

**POZOROVANIE:** Lítium je veľmi ľahký a mäkký kov, ktorý sa mnohými chemickými vlastnosťami podobá na horčík. Antityroidné účinky lítia sa najprv skúmali podrobne, keď sa zistilo, že pacienti s **psychiatrickým ochorením** liečeným lítium-karbonátom vyvinuli **hypotyreózu** a burcu. Štúdie na zvieratách a na ľuďoch následne odhalili, že lítium zvyšuje obsah intra-thyroidného jódu, inhibuje kondenzáciu jódotyrozínových zvyškov za vzniku jódtironínov (tyroxín (T4) a trijódtyronínu (T3)) a inhibuje uvoľňovanie T4 a T3.[https://www.uptodate.com/contents/lithium-and-the-thyroid#H2] Používa sa v prípade **ochorenia hypertireózy**. Usudzujeme, že tento atribút nám hovorí o tom, čí daný pacient podstupuje liečbu. Existujú len dva stavy. Buď pacient podstupuje liečbu alebo nepodstupuje. Preto budeme tento atribút považovat za **kategorický binárny atribút**, ktorý v sebe bude niesť hodnoty [true = podstupuje liečbu, false = nepodstupuje liečbu].

**KOREKCIA:** Prekonvertujeme si formát dát na boolean hodnoty

In [447]:
# Mapovanie True a False na hodnoty podľa toho ake si zvolíme
data['_lithium'] = data['lithium'].map({'t': True, 'f': False})
data['_lithium'].value_counts()

False    2786
True       14
Name: _lithium, dtype: int64

### Atribút - thyroid surgery

In [462]:
# Výpis prvých 5 záznamov pre daný atribút
data['thyroid surgery'].value_counts()

f    2761
t      39
Name: thyroid surgery, dtype: int64

**POZOROVANIE:** [https://youtu.be/0h5jM7eN6j4] Usudzujeme, že tento atribút nám hovorí o tom, čí sa daný pacient podstúpil operáciu. Existujú len dva stavy. Buď pacient podstúpil operáciu ale nepodstúpil. Preto budeme tento atribút považovat za **kategorický binárny atribút**, ktorý v sebe bude niesť hodnoty [true = podstúpil operáciu, false = nepodstúpil operáciu].

**KOREKCIA:** Prekonvertujeme si formát dát na boolean hodnoty

In [465]:
# Mapovanie True a False na hodnoty podľa toho ake si zvolíme
data['_thyroid_surgery'] = data['thyroid surgery'].map({'t': True, 'f': False})
data['_thyroid_surgery'].value_counts()

False    2761
True       39
Name: _thyroid_surgery, dtype: int64

## Ochorenia

### Atribút - sick

In [404]:
# Výpis prvých 5 záznamov pre daný atribút
data['sick'].value_counts()

f    2690
t     110
Name: sick, dtype: int64

**POZOROVANIE:** Usudzujeme, že tento atribút nám hovorí o tom, čí sa daný pacient je chorý. Existujú len dva stavy. Buď je pacient chorý alebo nie je chorý. Preto budeme tento atribút považovat za **kategorický binárny atribút**, ktorý v sebe bude niesť hodnoty [true = je chorý, false = nie je chorý].

**KOREKCIA:** Prekonvertujeme si formát dát na boolean hodnoty

In [427]:
# Mapovanie True a False na hodnoty podľa toho ake si zvolíme
data['_sick'] = data['sick'].map({'t': True, 'f': False})
data['_sick'].value_counts()

False    2690
True      110
Name: _sick, dtype: int64

### Atribút - goitre

In [450]:
# Výpis unikátnych hodnôt
data['goitre'].value_counts()

f    2775
t      25
Name: goitre, dtype: int64

**POZOROVANIE:** [https://youtu.be/TiBnSKyX2iU] Usudzujeme, že tento atribút nám hovorí o tom, čí sa daný trpí ochorením. Existujú len dva stavy. Buď pacient trpí ochorením alebo nie. Preto budeme tento atribút považovat za **kategorický binárny atribút**, ktorý v sebe bude niesť hodnoty [true = má ochorenie, false = nemá ochorenie].

**KOREKCIA:** Prekonvertujeme si formát dát na boolean hodnoty

In [451]:
# Mapovanie True a False na hodnoty podľa toho ake si zvolíme
data['_goitre'] = data['goitre'].map({'t': True, 'f': False})
data['_goitre'].value_counts()

False    2775
True       25
Name: _goitre, dtype: int64

### Atribút - tumor

In [452]:
# Výpis unikátnych hodnôt
data['tumor'].value_counts()

f    2729
t      71
Name: tumor, dtype: int64

**POZOROVANIE:** [https://www.endocrineweb.com/conditions/thyroid-cancer/thyroid-cancer] Usudzujeme, že tento atribút nám hovorí o tom, čí sa daný trpí ochorením. Existujú len dva stavy. Buď pacient trpí ochorením alebo nie. Preto budeme tento atribút považovat za **kategorický binárny atribút**, ktorý v sebe bude niesť hodnoty [true = má ochorenie, false = nemá ochorenie].

**KOREKCIA:** Prekonvertujeme si formát dát na boolean hodnoty

In [454]:
# Mapovanie True a False na hodnoty podľa toho ake si zvolíme
data['_tumor'] = data['tumor'].map({'t': True, 'f': False})
data['_tumor'].value_counts()

False    2729
True       71
Name: _tumor, dtype: int64

### Atribút - hypopituitary

In [458]:
# Výpis unikátnych hodnôt a ich počtov
data['hypopituitary'].value_counts()

f    2799
t       1
Name: hypopituitary, dtype: int64

**POZOROVANIE:** [https://www.emedicinehealth.com/hypopituitary/article_em.htm] Usudzujeme, že tento atribút nám hovorí o tom, čí sa daný trpí ochorením. Existujú len dva stavy. Buď pacient trpí ochorením alebo nie. Preto budeme tento atribút považovat za **kategorický binárny atribút**, ktorý v sebe bude niesť hodnoty [true = má ochorenie, false = nemá ochorenie]. V našom data sete sa nachádza iba jeden jediný pacient s takýmto ochorením. Tento pacient pravdepodobne len veľmi málo ovplyvní výledky budúceho trénovania a preto si myslíme, že by bolo vhodné tento atribút **zanedbať**.

**KOREKCIA:** Prekonvertujeme si formát dát na boolean hodnoty

In [457]:
# Mapovanie True a False na hodnoty podľa toho ake si zvolíme
data['_hypopituitary'] = data['hypopituitary'].map({'t': True, 'f': False})
data['_hypopituitary'].value_counts()

False    2799
True        1
Name: _hypopituitary, dtype: int64

### Atribút - psych

In [459]:
# Výpis unikátnych hodnôt a ich počtov
data['psych'].value_counts()

f    2665
t     135
Name: psych, dtype: int64

**POZOROVANIE:** [https://www.ncbi.nlm.nih.gov/pubmed/21996646] Usudzujeme, že tento atribút nám hovorí o tom, čí sa daný trpí ochorením. Existujú len dva stavy. Buď pacient trpí ochorením alebo nie. Preto budeme tento atribút považovat za **kategorický binárny atribút**, ktorý v sebe bude niesť hodnoty [true = má ochorenie, false = nemá ochorenie].

**KOREKCIA:** Prekonvertujeme si formát dát na boolean hodnoty

In [461]:
# Mapovanie True a False na hodnoty podľa toho ake si zvolíme
data['_psych'] = data['psych'].map({'t': True, 'f': False})
data['_psych'].value_counts()

False    2665
True      135
Name: _psych, dtype: int64

In [442]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2800 entries, 0 to 2799
Data columns (total 45 columns):
age                          2799 non-null float64
sex                          2800 non-null object
on thyroxine                 2800 non-null object
query on thyroxine           2800 non-null object
on antithyroid medication    2800 non-null object
sick                         2800 non-null object
pregnant                     2800 non-null object
thyroid surgery              2800 non-null object
I131 treatment               2800 non-null object
query hypothyroid            2800 non-null object
query hyperthyroid           2800 non-null object
lithium                      2800 non-null object
goitre                       2800 non-null object
tumor                        2800 non-null object
hypopituitary                2800 non-null object
psych                        2800 non-null object
TSH measured                 2800 non-null object
TSH                          2516 non-null

### Atribút - pregnant

In [428]:
# Výpis prvých 5 záznamov pre daný atribút
data['pregnant'].value_counts()

f    2759
t      41
Name: pregnant, dtype: int64

**POZOROVANIE:** Usudzujeme, že tento atribút nám hovorí o tom, či je daný pacient **tehotný**. Existujú len dva stavy. Buď je pacient tehotný alebo nie je tehotný. Preto budeme tento atribút považovat za **kategorický binárny atribút**, ktorý v sebe bude niesť hodnoty [true = je tehotný, false = nie je tehotný].

**KOREKCIA:** Prekonvertujeme si formát dát na boolean hodnoty

In [430]:
# Mapovanie True a False na hodnoty podľa toho ake si zvolíme
data['_pregnant'] = data['pregnant'].map({'t': True, 'f': False})
data['_pregnant'].value_counts()

False    2759
True       41
Name: _pregnant, dtype: int64

In [439]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2800 entries, 0 to 2799
Data columns (total 44 columns):
age                          2799 non-null float64
sex                          2800 non-null object
on thyroxine                 2800 non-null object
query on thyroxine           2800 non-null object
on antithyroid medication    2800 non-null object
sick                         2800 non-null object
pregnant                     2800 non-null object
thyroid surgery              2800 non-null object
I131 treatment               2800 non-null object
query hypothyroid            2800 non-null object
query hyperthyroid           2800 non-null object
lithium                      2800 non-null object
goitre                       2800 non-null object
tumor                        2800 non-null object
hypopituitary                2800 non-null object
psych                        2800 non-null object
TSH measured                 2800 non-null object
TSH                          2516 non-null

### Spojité atribúty (Kvantitatívne)
#### Intervalové
age, date_of_birth, hours-per-week <br/> 
#### Pomerové
TSH, T3, TT4, T4U, FTI, fnlwgt, capital-gain, capitital-loss <br/>
### Kategorické atribúty
#### Nominálne atribúty (hint. kategórie, stavy, mená, lable, tagy)
referral source - <br/>
class - <br/>
personal_info - <br/>
##### Binárne atribúty (hint. kategórie, stavy, mená, lable, tagy)
*Symetrické*<br/>
sex [M,F] - M = muž, F = žena<br/>
on thyroxine [f, t] - f = false, t = true  <br/>
sick [f, t] - f = false, t = true<br/>
pregnant [f, t] - f = false, t = true<br/>
thyroid surgery [f, t] - f = false, t = true<br/>
I131 treatment [f, t] - f = false, t = true<br/>
on antithyroid medication [f, t] - f = false, t = true<br/>
query hypothyroid [f, t] - f = false, t = true<br/>
lithium [f, t] - f = false, t = true<br/>
goitre [f, t] - f = false, t = true<br/>
tumor [f, t] - f = false, t = true<br/>
hypopituitary [f, t] - f = false, t = true<br/>
psych [f, t] - f = false, t = true<br/>
TSH measured [t, f] - f = false, t = true<br/>
T3 measured [t, f] - f = false, t = true<br/>
TT4 measured [t, f] - f = false, t = true<br/>
T4U measured [f, t] - f = false, t = true<br/>
FTI measured [f, t] - f = false, t = true<br/>
TBG measured [f, t] - f = false, t = true<br/>
#### Ordinálne
education [] - <br/>
education_num - <br/>

## Analýza atribútov po jednom (Univariate analysis)
V tejto časti sú vysvetlené a zobrazené vlastnosti každého atribútu po jednom.

## Analýza atribútov po pároch (Bivariate analysis)
V tejto časti sú vysvetlené a zobrazené vlastnosti atribútov, ktoré spolu môžu súvisieť.

### Spojité atribúty

### Kategorické atribúty